In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd gdrive/My Drive/
! git clone https://github.com/zhanghang1989/PyTorch-Encoding.git
%cd PyTorch-Encoding
! pip3 install torch==1.0.0
! pip3 install ninja
! python3 setup.py install

In [0]:
import torch
import encoding
import matplotlib.pyplot as plt
import numpy as np
import os
from glob import glob
from scipy.io import savemat

model_name = 'EncNet_ResNet101_PContext'
test_img_dir = 'test'
output_dir = 'output'
segs_str = 'segs'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

# Get the model
model = encoding.models.get_model(model_name, pretrained=True).cuda()
model.eval()

# Prepare the images
for index, img_path in enumerate(glob(os.path.join(test_img_dir, "*.jpg")), 1):
  print("\nImage %d" % index)
  print("Running code for %s\n" % img_path)
  img = encoding.utils.load_image(img_path).cuda().unsqueeze(0)
  output = model.evaluate(img)
  # Make prediction
  predict = (torch.max(output, 1)[1].cpu().numpy() + 1)[0]

  set_unique = set(list(np.reshape(predict, -1)))
  
  for index, el in enumerate(set_unique):
    predict = np.where(predict == el, index, predict) 
  
  img_base_name = os.path.basename(img_path)
  img_name = os.path.splitext(img_base_name)[0]
  
  obj_arr = np.empty((1,), dtype=np.object)
  obj_arr[0] = predict
  savemat(os.path.join(output_dir, "%s.mat" % img_name), {segs_str: obj_arr})

# print(set(list(np.reshape(predict, -1))))
# plt.imsave('output_bird.png', predict)